In [2]:
import pandas as pd
import openai
import time
import os
from tqdm import tqdm
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
def setup_openai():
    """OpenAIクライアントの設定"""
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        raise ValueError("OPENAI_API_KEY環境変数が設定されていません")
    openai.api_key = api_key
    return openai

def ask_gpt_contrary(assumption, proposition, client):
    """
    GPT-4o-miniに対してContrary判定を問い合わせる関数
    
    Args:
        assumption (str): 仮定文
        proposition (str): 命題文
        client: OpenAIクライアント
    
    Returns:
        str: "True" または "False"
    """
    
    prompt = f"""
    You are an expert in logical reasoning and hotel review analysis.
    
    Your task is to determine if phrase B is a contrary of phrase A in the context of hotel reviews.
    A contrary relationship means that the two phrases express opposite or contradictory concepts.
    
    Here are some examples:
    
    Example 1:
    - Phrase A (Assumption): "no_evident_not_clean_room"
    - Phrase B (Proposition): "loud_air_conditioner"
    - Answer: False (not contrary - different aspects)
    
    Example 2:
    - Phrase A (Assumption): "no_evident_not_clean_room"
    - Phrase B (Proposition): "loud_people_in_next_hotel"
    - Answer: False (not contrary - different aspects)
    
    Example 3:
    - Phrase A (Assumption): "no_evident_not_clean_room"
    - Phrase B (Proposition): "small_room"
    - Answer: False (not contrary - different aspects)
    
    Example 4:
    - Phrase A (Assumption): "no_evident_not_clean_room"
    - Phrase B (Proposition): "hard_mattress"
    - Answer: False (not contrary - different aspects)
    
    Example 5:
    - Phrase A (Assumption): "no_evident_not_clean_room"
    - Phrase B (Proposition): "hair_on_floor"
    - Answer: True (contrary - evidence of uncleanliness contradicts no evidence of unclean room)
    
    Example 6:
    - Phrase A (Assumption): "no_evident_not_clean_room"
    - Phrase B (Proposition): "need_better_cleaner"
    - Answer: True (contrary - needing better cleaning contradicts no evidence of unclean room)
    
    Now analyze this pair:
    - Phrase A (Assumption): "{assumption}"
    - Phrase B (Proposition): "{proposition}"
    
    Question: Is phrase B a contrary of phrase A?
    
    Please answer with only "True" or "False".
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a logical reasoning expert. Answer only with 'True' or 'False'."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10,
            temperature=0
        )
        
        answer = response.choices[0].message.content.strip()
        # 答えが"True"か"False"でない場合のフォールバック
        if answer.lower() == "true":
            return "True"
        elif answer.lower() == "false":
            return "False"
        else:
            print(f"Unexpected response: {answer}")
            return "False"  # デフォルトはFalse
            
    except Exception as e:
        print(f"Error calling OpenAI API: {e}")
        return "False"  # エラー時はFalse


In [7]:
def process_csv_with_gpt(input_file_path, output_file_path, sample_size=None):
    """
    CSVファイルを読み込んでGPT-4o-miniでContrary判定を行い、結果を保存
    
    Args:
        input_file_path (str): 入力CSVファイルのパス
        output_file_path (str): 出力CSVファイルのパス
        sample_size (int, optional): 処理するサンプル数（Noneの場合は全て処理）
    """
    
    print("CSVファイルを読み込み中...")
    df = pd.read_csv(input_file_path)
    
    if sample_size:
        df = df.head(sample_size)
        print(f"サンプル {sample_size} 件を処理します")
    else:
        print(f"全 {len(df)} 件を処理します")
    
    print("OpenAIクライアントを設定中...")
    client = setup_openai()
    
    # isContraryカラムを初期化
    df['isContrary'] = ""
    
    print("GPT-4o-miniへの問い合わせを開始...")
    
    for i, (index, row) in enumerate(tqdm(df.iterrows(), total=len(df), desc="Processing")):
        assumption = row['Assumption']
        proposition = row['Proposition']
        
        # GPT-4o-miniに問い合わせ
        result = ask_gpt_contrary(assumption, proposition, client)
        df.at[index, 'isContrary'] = result
        
        # API制限を避けるために少し待機
        time.sleep(0.5)
        
        # 進捗を定期的に保存（50件ごと）
        if (i + 1) % 50 == 0:
            df.to_csv(output_file_path, index=False)
            print(f"進捗保存: {i + 1} 件完了")
    
    # 最終結果を保存
    df.to_csv(output_file_path, index=False)
    print(f"処理完了！結果を {output_file_path} に保存しました")
    
    return df


In [8]:
# 実行セル - テストモード（10件）
input_file = "../data/RoomSilver_ContP_BodyN.csv"
output_file = "../data/RoomSilver_ContP_BodyN_test_sample.csv"

print("=== GPT-4o-mini Contrary Analysis (テストモード) ===")
print(f"入力ファイル: {input_file}")
print(f"出力ファイル: {output_file}")

# テスト実行（10件のサンプル）
result_df = process_csv_with_gpt(input_file, output_file, sample_size=100)

# 結果の要約を表示
print("\n=== 結果要約 ===")
print(f"処理済み件数: {len(result_df)}")
print(f"True の件数: {(result_df['isContrary'] == 'True').sum()}")
print(f"False の件数: {(result_df['isContrary'] == 'False').sum()}")

# 結果を表示
print("\n=== 結果データ ===")
result_df


=== GPT-4o-mini Contrary Analysis (テストモード) ===
入力ファイル: ../data/RoomSilver_ContP_BodyN.csv
出力ファイル: ../data/RoomSilver_ContP_BodyN_test_sample.csv
CSVファイルを読み込み中...
サンプル 100 件を処理します
OpenAIクライアントを設定中...
GPT-4o-miniへの問い合わせを開始...


Processing:  50%|█████     | 50/100 [01:01<00:51,  1.04s/it]

進捗保存: 50 件完了


Processing: 100%|██████████| 100/100 [01:56<00:00,  1.17s/it]

進捗保存: 100 件完了
処理完了！結果を ../data/RoomSilver_ContP_BodyN_test_sample.csv に保存しました

=== 結果要約 ===
処理済み件数: 100
True の件数: 8
False の件数: 92

=== 結果データ ===


,ID,Assumption,Proposition,Unnamed: 3,isContrary
0,1,no_evident_not_clean_room,can_hear_airplane_coming,NaN,False
1,2,no_evident_not_clean_room,can_hear_airplane_leaving,NaN,False
2,3,no_evident_not_clean_room,not_enough_drawer,NaN,False
3,4,no_evident_not_clean_room,fridge_could_be_put_on_beforehand,NaN,False
4,5,no_evident_not_clean_room,shower_took_a_while_to_get_hot,NaN,False
...,...,...,...,...,...
95,96,no_evident_not_clean_room,bathroom_floor_sink_closed_with_tape,NaN,False
96,97,no_evident_not_clean_room,bug_under_bathroom_floor_sink,NaN,True
97,98,no_evident_not_clean_room,only_3_channel_TV,NaN,False
98,99,no_evident_not_clean_room,Netflix_difficult_to_sign_out,NaN,False


In [ ]:
# 全データ処理用セル - 実行時は注意！（データが大きく時間がかかります）
# 以下のコメントアウトを外して実行してください

# input_file = "../data/RoomSilver_ContP_BodyN.csv"
# output_file = "../data/RoomSilver_ContP_BodyN_with_isContrary.csv"

# print("=== GPT-4o-mini Contrary Analysis (全データ処理) ===")
# print("警告: 全データを処理します。時間がかかる可能性があります。")
# print(f"入力ファイル: {input_file}")
# print(f"出力ファイル: {output_file}")

# # 全データ実行
# result_df_full = process_csv_with_gpt(input_file, output_file)

# # 結果の要約を表示
# print("\n=== 最終結果要約 ===")
# print(f"処理済み件数: {len(result_df_full)}")
# print(f"True の件数: {(result_df_full['isContrary'] == 'True').sum()}")
# print(f"False の件数: {(result_df_full['isContrary'] == 'False').sum()}")

print("全データ処理を実行するには、上記のコメントアウトを外してください")
